## INFO284 - Group exam

Your tasks are:

#### Preprocessing: 
Get to understand the data and remove columns and rows that you do not find useful for your machine learning models. It is possible to for example focus on one type of gear, one or a few species, group categories of species, etc. etc. This will help to reduce
the data set you are working on. Understanding data and preparing them for model building
is a main task of machine learning.

#### Supervised learning: 
Build at least three machine learning models to predict or classify catch
data related to a fishery operation, i.e. species caught, amounts of one fish species (sums of amounts is also possible), or other features related to fish catches. One of the models needs to be a deep learning model.

#### Unsupervised learning: 
Build a clustering model for the data set. You may use a different preprocessing for this task than for the data in task b).


You shall deliver code in the form of a well commented Jupyter notebook. This code needs to run on the original data set, so any preprocessing you choose to do needs to be programmed in Python and included in the notebook. The code shall in the end return the results of your experiments with your chosen models. You need to explain

- Important and relevant properties of the data
- how you preprocessed data like which features you selected, did you do dimension reduction 
- how you reformatted data, etc.
- how you decided on parameters for your machine learning models,
- if you used any regularization techniques? In case how.
- how the methods were measured and compared

Please inform about any special Python libraries that need to be installed to make your code runnable.

Finally, as a concluding comment in the Jupyter notebook, you need to write a summary of your results, and discuss consequences of such results.

It is not necessarily so that high scores for machine learning models will give a good grade on your report, or vice versa, low scores a bad grade. What counts is a well-argued, well described, and smart machine learning investigation from start to end. The problem may in fact be of such nature that it is not possible to get really good results on these data.
Final note: These data are prepared for this course and are shared with you in confidence that you do not share them in any way but use them only for the purpose of this exam.


## Importerer nødvendige biblioteker
Leser csv.

In [91]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor

df = pd.read_csv("GroupExam2024_data_and_documentation/elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv", sep=';', decimal=',')

In [92]:
train, test = train_test_split(df, test_size = 0.15)

In [124]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45816 entries, 166855 to 114007
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Melding ID                45816 non-null  int64  
 1   Meldingstidspunkt         45816 non-null  object 
 2   Meldingsdato              45816 non-null  object 
 3   Meldingsklokkeslett       45816 non-null  object 
 4   Starttidspunkt            45816 non-null  object 
 5   Startdato                 45816 non-null  object 
 6   Startklokkeslett          45816 non-null  object 
 7   Startposisjon bredde      45816 non-null  float64
 8   Startposisjon lengde      45816 non-null  float64
 9   Hovedområde start (kode)  45521 non-null  float64
 10  Hovedområde start         45185 non-null  object 
 11  Lokasjon start (kode)     45521 non-null  float64
 12  Havdybde start            45816 non-null  int64  
 13  Stopptidspunkt            45816 non-null  object 
 14  Stopp

Av df.info ser vi de ulike kolonnene og hvilken datatype de er. og hvor mange "non-null" verdier som eksisterer. Kan se her at det eksisterer null-verdier i de fleste kolonner. Dette renses senere. Konverterer datatype etter å ha filtrert ut de uaktuelle kolonnene.

In [94]:
train.head()

,Melding ID,Meldingstidspunkt,Meldingsdato,Meldingsklokkeslett,Starttidspunkt,Startdato,Startklokkeslett,Startposisjon bredde,Startposisjon lengde,Hovedområde start (kode),...,Art - FDIR,Art - gruppe (kode),Art - gruppe,Rundvekt,Lengdegruppe (kode),Lengdegruppe,Bruttotonnasje 1969,Bruttotonnasje annen,Bredde,Fartøylengde
0,1497177,01.01.2018,01.01.2018,00:00,31.12.2017,31.12.2017,00:00,-60.350,-46.133,NaN,...,Antarktisk krill,506.0,Antarktisk krill,706714.0,5.0,28 m og over,9432.0,NaN,19.87,133.88
1,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,74.885,16.048,20.0,...,Hyse,202.0,Hyse,9594.0,5.0,28 m og over,1476.0,NaN,12.60,56.80
2,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,74.885,16.048,20.0,...,Torsk,201.0,Torsk,8510.0,5.0,28 m og over,1476.0,NaN,12.60,56.80
3,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,74.885,16.048,20.0,...,Blåkveite,301.0,Blåkveite,196.0,5.0,28 m og over,1476.0,NaN,12.60,56.80
4,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,74.885,16.048,20.0,...,Sei,203.0,Sei,134.0,5.0,28 m og over,1476.0,NaN,12.60,56.80


Basert på datasettet og kolonnene vi har sett på så ønsker vi å predikere rundvekt av en aktivitet.


Ser av datasettet hvordan dataen er strukturert. 
**Organisering av datasettet**
Hver rad representerer en fangst av art. Flere rader er registrert med samme starttidspunkt eller stopptidspunkt. De ulike fangstene som er gjort på samme tidspunkt med samme melding ID kommer fra samme fiskeaktivitet. Videre så har de ulike aktivitetene som er gjort på samme Melding ID samme meldingsdato, og kommer fra samme båt.

**Bruttotonnasje 1969 og Bruttotonnasje annen**
Vi ser også at kolonnen Bruttotonnasje annen og bruttotonnasje 1969 ser ut til å utfylle hverandre.Med bruttotonnasje menes det totale volumet av hulrom i båten. Det kan ikke helt sammenlignes med lastekapasitet, men kan være en indikator. Bruttotonnasje 1969 er standarden i dag for registrering av dette målet av båten. Bruttotonnasje annen viser de båtene som er registrert gjennom andre systemer enn standarden.

Vi antar at selv om det antakelig er forskjeller i hvordan disse båtene har målt sin Bruttotonnasje, så er det sannsynligvis også så mange likheter at det ikke har betydning. Dette er det kanskje nødvendig å undersøke nermere dersom man vil ha en mer nøyaktig analyse av dataene. Disse to kolonnene vil vi derfor slå sammen. til en felles kolonne.

Hva leter vi etter?
Siden dette er en samling av fiskeridata, så tenker vi det er interessant å se nærmere på fangstdataene. Her kunne vi hatt mange interessante innfallsvinkler, men vi ønsker å se om det er mulig å predikere rundvekt av en bestemt art gitt den informasjonen vi har. Siden fiskere bruker ulike redskaper for å fange fisk kan vi kan anta at noen redskaper og metoder er mer egnet for å fange bestemte arter. For eksempel så brukes ikke teiner for å fange torsk eller trål for å fange makrell (Må sjekke opp i). I tillegg vet vi også at det foregår fiske etter ulike arter på ulike steder og tidspunkt. Skreifiske for eksempel foregår i bestemte tidsrom etter bestemte arter, med bestemte redskaper. Videre kan vi anta at det samme gjelder for for eksempel sildefiske. Kunnskap om hvordan, hvor og hva som fiskes etter er derfor viktig for å forstå datasettet. Videre avsnitt bruker vi på å undersøke informasjon omkring fisket.


In [95]:
# Først undersøker vi hvor mange registreringer det er av de ulike fisketypene og hvor mye det fiskes av hver.

species_weight_sum = df.groupby('Art - gruppe')['Rundvekt'].sum().sort_values(ascending=False)
print(df['Art - gruppe'].value_counts())
print(species_weight_sum)

Art - gruppe
Torsk                                       56574
Annen torskefisk                            45286
Sei                                         42557
Hyse                                        39120
Annen flatfisk, bunnfisk og dypvannsfisk    25267
Uer                                         19681
Steinbiter                                  16181
Dypvannsreke                                13678
Blåkveite                                    8046
Snøkrabbe                                    6070
Sild, annen                                  6006
Skater og annen bruskfisk                    4067
Makrell                                      3013
Kolmule                                      2843
Øyepål                                       2210
Annen pelagisk fisk                          1802
Vassild og strømsild                         1650
Andre skalldyr, bløtdyr og pigghuder         1329
Lodde                                        1188
Tobis og annen sil                   

Her ser vi en hovedvekt av fiske etter Torsk. På en annenplass kommer sei. Reke er det også mye aktivitet rundt.

In [96]:
# Gjør samme undersøkelse av reskaper.

redskap_weight_sum = df.groupby('Redskap FDIR')['Rundvekt'].sum().sort_values(ascending=False)
print(df['Redskap FDIR'].value_counts())
print(redskap_weight_sum)

Redskap FDIR
Bunntrål                                  120487
Snurrevad                                  51615
Andre liner                                43992
Reketrål                                   22421
Udefinert garn                             19277
Udefinert trål                             15870
Snurpenot/ringnot                          12711
Teiner                                      6818
Bunntrål par                                3335
Dobbeltrål                                  2836
Flytetrål                                   2732
Flytetrål par                               1512
Settegarn                                    649
Harpun og lignende uspesifiserte typer       494
Annet                                        462
Juksa/pilk                                    19
Dorg/harp/snik                                16
Name: count, dtype: int64
Redskap FDIR
Snurpenot/ringnot                         776961216.0
Flytetrål                                 468684685.0
Bunntrå

Her ser vi at de to mest brukte fangstredskapene er Bunntrål og Snurrevad. Videre er andre liner, reketrål og andre liner høyt oppe på listen. fra fiskeridirektoratet (kilde) vet vi at Bunntrål og Snurrevad er garn/not som blir slept langs bunnen av sjøen og samler opp fisk på den måten. Andre liner (dersom vi har forstått det riktig) er fiske med agn og krok som gjerne slepes bak båten. 

Det som er naturlig å undersøke nå er hvordan sammensetningen av fisk er med de ulike fangsmetodene, og hvor det er vanlig at de blir brukt.

In [97]:
# Ser etter hvilke fisk som blir fisket med Bunntrål:
condition = df[df['Redskap FDIR'] == 'Bunntrål']
species_weight_sum = condition.groupby('Art - gruppe')['Rundvekt'].sum().sort_values(ascending=False)
print(condition['Art - gruppe'].value_counts(), species_weight_sum)

Art - gruppe
Torsk                                       23366
Sei                                         18248
Hyse                                        16632
Annen torskefisk                            14959
Uer                                         14252
Annen flatfisk, bunnfisk og dypvannsfisk     8621
Dypvannsreke                                 7440
Steinbiter                                   7281
Blåkveite                                    5103
Skater og annen bruskfisk                     805
Øyepål                                        721
Andre skalldyr, bløtdyr og pigghuder          610
Kolmule                                       485
Vassild og strømsild                          479
Annen pelagisk fisk                           402
Sild, annen                                   320
Makrell                                       231
Tobis og annen sil                            179
Haifisk                                        64
Lodde                                

In [98]:
# Ser etter hvilke fisk som blir fisket med Snurrevad:
condition = df[df['Redskap FDIR'] == 'Snurrevad']

species_weight_sum = condition.groupby('Art - gruppe')['Rundvekt'].sum().sort_values(ascending=False)
print(condition['Art - gruppe'].value_counts(), species_weight_sum)

Art - gruppe
Torsk                                       16347
Hyse                                        11950
Sei                                         11259
Annen flatfisk, bunnfisk og dypvannsfisk     4185
Annen torskefisk                             4124
Steinbiter                                   1282
Uer                                           811
Blåkveite                                     367
Sild, annen                                    30
Makrell                                        26
Skater og annen bruskfisk                      11
Dypvannsreke                                   10
Lodde                                           9
Andre skalldyr, bløtdyr og pigghuder            4
Haifisk                                         4
Sjøpattedyr                                     1
Annen pelagisk fisk                             1
Name: count, dtype: int64 Art - gruppe
Torsk                                       57885810.0
Sei                                        

In [99]:
# Ser etter hvilke fisk som blir fisket med andre liner:
condition = df[df['Redskap FDIR'] == 'Andre liner']
species_weight_sum = condition.groupby('Art - gruppe')['Rundvekt'].sum().sort_values(ascending=False)
print(condition['Art - gruppe'].value_counts(), species_weight_sum)

Art - gruppe
Annen torskefisk                            11202
Torsk                                        7372
Steinbiter                                   6712
Hyse                                         6641
Uer                                          3236
Sei                                          2543
Annen flatfisk, bunnfisk og dypvannsfisk     2315
Blåkveite                                    2105
Skater og annen bruskfisk                    1786
Haifisk                                        58
Kongekrabbe, annen                              4
Brunalger                                       1
Name: count, dtype: int64 Art - gruppe
Torsk                                       33155234.0
Hyse                                        24829551.0
Annen torskefisk                            21104924.0
Steinbiter                                   4769635.0
Blåkveite                                    3210210.0
Sei                                           914214.0
Skater og annen br

In [100]:
# Ser etter hvilke fisk som blir fisket med reketrål:
condition = df[df['Redskap FDIR'] == 'Reketrål']
species_weight_sum = condition.groupby('Art - gruppe')['Rundvekt'].sum().sort_values(ascending=False)
print(condition['Art - gruppe'].value_counts(), species_weight_sum)

Art - gruppe
Dypvannsreke                                5838
Annen flatfisk, bunnfisk og dypvannsfisk    3968
Annen torskefisk                            3297
Torsk                                       2950
Sei                                         2769
Skater og annen bruskfisk                   1227
Hyse                                         497
Øyepål                                       461
Andre skalldyr, bløtdyr og pigghuder         338
Annen pelagisk fisk                          190
Uer                                          182
Steinbiter                                   177
Blåkveite                                    127
Kolmule                                      119
Vassild og strømsild                         112
Sild, annen                                   29
Makrell                                       26
Haifisk                                       19
Tobis og annen sil                             8
Mesopelagisk fisk                              2
Brunalg

In [101]:
# Ser etter hvilke fisk som blir fisket med snurpenot, ringnot et fartøy:
condition = df[df['Redskap FDIR'] == 'Snurpenot/ringnot']
species_weight_sum = condition.groupby('Art - gruppe')['Rundvekt'].sum().sort_values(ascending=False)
print(condition['Hovedart FAO'].value_counts(), species_weight_sum)

Hovedart FAO
Sild                    5539
Sei                     1279
Lodde                   1250
Makrell                  952
Torsk                    735
Brisling                 171
Tobis og annen sil       144
Hestmakrell               98
Kolmule                   64
Strømsild/Vassild         23
Strømsild                 20
Hyse                       7
Vassild                    5
Blåkveite                  5
Makrellstørje              2
Nordlig lysprikkfisk       1
Name: count, dtype: int64 Art - gruppe
Sild, annen                                 364481891.0
Lodde                                       177152131.0
Makrell                                     160745614.0
Sei                                          34768481.0
Kolmule                                      24500480.0
Tobis og annen sil                            5694001.0
Torsk                                         4394568.0
Kystbrisling                                  2688800.0
Annen pelagisk fisk                 

Etter å ha studert de ulike redskapene og hva som blir fisket med de aktuelle, ser vi også konturene av hvordan de blir brukt. Bunntrål er den mest ukritiske fangstmetoden og den tar med seg et større mangfold av fisk enn for eksempel snurrevad som kanskje er naturlig å sammenligne med. Man skulle tro av navnet at en reketrål var mer målrettet i fangsten, og at andelen reker var større i denne sammensetningen, men her kan vi også anta, at siden reker står ganske lavt i næringskjeden, så vil det være naturlig at det er en stor andel av "rovfisker" i miksen. Vi har ikke sett her hvor stor andel av vekten de andre fiskene utgjør

Redskaper:
    Snurpenot/ringnot:
        Dette er det redskapet som det blir fisket desidert mest fisk med, selv om det er mye flere registreringer av andre redskaper. Forklaring her kan være at at med Snurpenot/ringnot blir det fisket mest stimfisk som sild, lodde, makrell osv, som opptrer i stim og dermed er naturlig at man fanger mye av når man først har funnet stimen. Det trenger ikke nødvendigvis bety at det er et mer effektivt fiske.
    Bunntrål:
        Med dette redskapet blir det fisket mest torsk og sei, men også ganske mye av andre arter. Det er det redskapet som vi har undersøkt som har jevnest fordeling arter, som betyr at det er det redskapet som bidrar med mest bifangst.
    Snurrevad:
        Dette redskapet ligner på bunntrål, men det virker som det drives litt mer målrettet fiske med dette. Spredningen av fangst av forskjellige arter er ikke så stor.
    Andre liner:
    Reketrål:
    
Basert på denne infoen er det interessant å dykke videre inn i et bestemt redskap og art og muligens et bestemt område. 


In [102]:
print(df['Hovedområde start'].value_counts())

Hovedområde start
Eigersundbanken                  45349
Røstbanken til Malangsgrunnen    29099
Vest-Finnmark                    27711
Øst-Finnmark                     26995
Bjørnøya                         25265
Shetland                         24813
Nordkappbanken                   18635
Vikingbanken                     16600
Skagerrak                        16097
Storegga-Frøyabanken             14884
Helgelandsbanken                  8591
Sentralbanken                     5584
Storbanken                        4823
Thor Iversens Bank                4551
Vestfjorden (Lofoten)             4469
Sentrale Nordsjø                  3387
Hopen                             3161
Vest av Skottland (Hebridene)     2008
Sørlige Norskehav                 1990
Østlige Norskehav                 1936
Admiralityfeltet                  1909
Skolpenbanken                     1904
Gåsebanken                        1699
Storfjord/Hinlopenstredet         1363
Vest-Spitsbergen                  1242
Vest av

Lokasjon:
Det er ingen hemmelighet at fiske etter ulike arter foregår på ulike steder til ulikt tidspunkt, avhengig av hva man ønsker å fiske, og hvor mye. For eksempel har skreifiske i lofoten pågått i tusener av år. Storsilden har også tradisjonelt sett kommet inn til vestlandskysten på vinteren.
https://open-data-fiskeridirektoratet-fiskeridir.hub.arcgis.com/datasets/1b8c3425aaf643e9b2abd3bc70111697/explore

Sei:
Vi har valgt å fokusere på seifiske: I Norge er det hovedsaklig fiske etter to grupper av sei. En gruppe som beiter i havområdene mellom Norge og Skottland https://www.hi.no/hi/temasider/arter/sei-i-nordsjoen og en annen som beiter i kystområdene fra vestlandet til finnmark. https://www.hi.no/hi/temasider/arter/nordaustarktisk-sei.
De er rovfisker og vandrefisk og kan følge sild langt ut i norskehavet. 

Om vi skal fokusere på den nordøstarktiske seien, kan det se ut fra denne informasjon at det kan være lurt å fokusere på hovedområdene fra 0-7 minus 2, samt 28. Om vi skal inkludere Nordsjøseien, så kan vi inkludere områdene 40-43, men da beveger vi oss også litt utenfor de de norske områdene.

Videre er det 



In [103]:
df.loc[:, 'Bruttotonnasje kombinert'] = df['Bruttotonnasje 1969'].fillna(df['Bruttotonnasje annen'])

# Dropper kolonner: Meldingstids
dropped_df = df.drop(columns=[
    #'Melding ID', Trengs muligens for å skille ulike tokter??
    'Meldingstidspunkt',  # Trenger ikke tidspunkt for melding når vi undersøker selve fisket. 
    'Meldingsdato',
    'Meldingsklokkeslett',
    'Starttidspunkt', 
    'Startdato', 
    'Startklokkeslett',
    'Startposisjon bredde', 
    'Startposisjon lengde',
    #'Hovedområde start (kode)', 
    'Hovedområde start',
    'Lokasjon start (kode)',
    'Havdybde start',
    #'Stopptidspunkt', # Det kan være interessant å se om det er sammenheng i tidspunkt på døgn og fangst.
    'Stoppdato', # All tidspunkt informasjon får vi i kolonnen over.
    'Stoppklokkeslett',
    'Varighet',
    'Fangstår', # Irrelevant når så og si all data er fra 2018
    #'Stopposisjon bredde', 
    #'Stopposisjon lengde',
    'Hovedområde stopp (kode)', 
    'Hovedområde stopp',
    'Lokasjon stopp (kode)',
    #'Havdybde stopp', 
    #'Trekkavstand',
    'Redskap FAO (kode)',
    'Redskap FAO',
    'Redskap FDIR (kode)',
    #'Redskap FDIR', 
    #'Hovedart FAO (kode)', 
    #'Hovedart FAO',
    'Hovedart - FDIR (kode)',
    #'Art FAO (kode)', 
    #'Art FAO',
    'Art - FDIR (kode)',
    'Art - FDIR',
    'Art - gruppe (kode)',
    #'Art - gruppe', # Kan være nyttig for å få oversikt
    #'Rundvekt', 
    'Lengdegruppe (kode)', # Vi har valgt å bruke Bruttotonnasje som størrelse/kapasitetmål på båtene. 
    'Lengdegruppe',
    #'Bruttotonnasje kombinert'
    'Bruttotonnasje 1969',
    'Bruttotonnasje annen',
    'Bredde',
    'Fartøylengde'
])
dropped_df.head()

,Melding ID,Hovedområde start (kode),Stopptidspunkt,Stopposisjon bredde,Stopposisjon lengde,Havdybde stopp,Trekkavstand,Redskap FDIR,Hovedart FAO (kode),Hovedart FAO,Art FAO (kode),Art FAO,Art - gruppe,Rundvekt,Bruttotonnasje kombinert
0,1497177,NaN,01.01.2018,-60.400,-46.150,-805,5649.0,Flytetrål,KRI,Antarktisk krill,KRI,Antarktisk krill,Antarktisk krill,706714.0,9432.0
1,1497178,20.0,31.12.2017 04:16,74.914,15.969,-334,3970.0,Bunntrål,HAD,Hyse,HAD,Hyse,Hyse,9594.0,1476.0
2,1497178,20.0,31.12.2017 04:16,74.914,15.969,-334,3970.0,Bunntrål,HAD,Hyse,COD,Torsk,Torsk,8510.0,1476.0
3,1497178,20.0,31.12.2017 04:16,74.914,15.969,-334,3970.0,Bunntrål,HAD,Hyse,GHL,Blåkveite,Blåkveite,196.0,1476.0
4,1497178,20.0,31.12.2017 04:16,74.914,15.969,-334,3970.0,Bunntrål,HAD,Hyse,POK,Sei,Sei,134.0,1476.0


In [104]:
# Lager en betingelse med aktuelle områder.
northeastern = [0, 1, 3, 4, 5, 6, 7, 28]
northeastern_ocean = [30, 34, 37, 39, 12, 36] # For å undersøke om det er noe særlig seifiske utenfor de mer kystnære strøk
northsea = [40, 41, 42, 43] # For å undersøke seifiske
areas_condition = dropped_df['Hovedområde start (kode)'].isin(northeastern)
# Lager en betingelse med aktuelle fiskearter
fish_types_conditions = dropped_df['Hovedart FAO'] == 'Sei'
# Lager en betingelse med aktuelle redskaper:
fish_tools_conditions = dropped_df['Redskap FDIR'] == 'Bunntrål'
conditions = areas_condition & fish_types_conditions & fish_tools_conditions
filtered_df = dropped_df[conditions]
filtered_df.describe()  # Får her et inntrykk av ekstremverdier og gjennomsnitt.

,Melding ID,Hovedområde start (kode),Stopposisjon bredde,Stopposisjon lengde,Havdybde stopp,Trekkavstand,Rundvekt,Bruttotonnasje kombinert
count,2.703000e+04,27030.000000,27030.000000,27030.000000,27030.000000,27030.000000,27030.000000,27030.000000
mean,1.646568e+06,8.622494,66.726095,13.021517,-192.309989,13474.779948,2676.027340,1899.467999
std,9.436028e+04,8.318941,4.139766,8.355615,73.043725,11594.323343,5357.860741,1103.258222
min,1.497341e+06,0.000000,59.647000,1.654000,-1276.000000,0.000000,0.000000,129.000000
25%,1.549231e+06,4.000000,62.836000,5.339000,-227.000000,4990.000000,77.000000,1194.000000
50%,1.665665e+06,5.000000,68.030000,10.942000,-183.000000,10145.000000,260.000000,1572.000000
75%,1.729002e+06,7.000000,70.882000,20.581000,-145.000000,19322.000000,2377.500000,2580.000000
max,1.800286e+06,28.000000,72.060000,32.226000,-49.000000,233749.000000,86506.000000,4427.000000


Datacleaning:
Nullverdier.

Av df.describe leser vi først max og min verdier for å se om det er noen ekstreme eller usannsynlige verdier.
Rundvekt, 0 - registreringer. Ser at det er registreringer med verdien 0 i trekkavstand og rundvektkolonnen. Vi anser dette som feilregistreringer. Av trålens natur, at den soper store havområder, ser vi det som lite trolig at rundvekt 0 skal forekomme. Videre kan vi anta at registreringer med 0 i trekkavstand også er lite sannsynlig. 

Trekkavstand:
Vi ser at det ligger registreringer på over 200000 meter, det vil si 200 km eller 20 mil. Dette er store avstander for en trål som vanligvis opererer i snittfart på mellom 3,4 (6,3km/h) og 5 knop (9,26 km/h). I en snittfart på 10km/h så vil da trålen bruke over 20 timer. Med så lang varighet vil det være svært ugunstige forhold for fisken. 
Vi ser også at 5% av registreringene er over 35446m. Her ser vi også at med relativt høy fart, vil aktiviteten ha varighet på nærmere 4 timer. Ved å kutte registreringer over 50000 gir det oss en god margin på mulige feilregistreringer.

Dybde:
Vi ser det er registrert positive havdybde-verdier, i tillegg er det også registrert aktiviteter på -1276 dybde. Men de aller fleste aktivitetene er registrert mellom -227 og -145 meter. Fiskeridirektoratet beskriver at "Sei førekjem både pelagisk og som botnfisk, frå 0 til 300 meters djup." (https://www.hi.no/hi/temasider/arter/nordaustarktisk-sei) Vi kan anta at dette er enten en feilregistrering eller at fangsten er gjort når trålen er på vei opp av dypet. 



In [105]:
trekkavstand = filtered_df['Trekkavstand'] == 0
filtered_df[trekkavstand].describe()

,Melding ID,Hovedområde start (kode),Stopposisjon bredde,Stopposisjon lengde,Havdybde stopp,Trekkavstand,Rundvekt,Bruttotonnasje kombinert
count,4.700000e+01,47.000000,47.000000,47.000000,47.000000,47.0,47.000000,47.000000
mean,1.617684e+06,6.978723,67.872128,14.956021,-247.638298,0.0,1747.212766,967.531915
std,1.209650e+05,6.585522,3.445144,8.156399,131.462414,0.0,2682.257841,695.498284
min,1.497435e+06,3.000000,61.072000,2.499000,-751.000000,0.0,8.000000,129.000000
25%,1.511569e+06,4.500000,63.284000,6.134000,-235.000000,0.0,136.500000,496.000000
50%,1.535772e+06,5.000000,69.450000,15.867000,-196.000000,0.0,547.000000,691.000000
75%,1.730467e+06,7.000000,70.205000,18.816000,-177.000000,0.0,2424.000000,1498.000000
max,1.799900e+06,28.000000,71.238000,31.306000,-168.000000,0.0,13525.000000,2580.000000


Ser av trekkavstand at det i 0-registreringene av trekkavstand også fanges fisk. Dette gir ikke mening med tanke på at det følgelig må trekkes en viss avstand for å få fisk.

In [106]:
rundvekt = filtered_df['Rundvekt'] == 0
filtered_df[rundvekt].head()

,Melding ID,Hovedområde start (kode),Stopptidspunkt,Stopposisjon bredde,Stopposisjon lengde,Havdybde stopp,Trekkavstand,Redskap FDIR,Hovedart FAO (kode),Hovedart FAO,Art FAO (kode),Art FAO,Art - gruppe,Rundvekt,Bruttotonnasje kombinert
7538,1503695,5.0,11.01.2018 08:35,70.694,17.278,-282,29217.0,Bunntrål,POK,Sei,GHL,Blåkveite,Blåkveite,0.0,3104.0
9677,1505102,4.0,14.01.2018 20:42,70.569,17.249,-276,5359.0,Bunntrål,POK,Sei,MZZ,Annen marin fisk,"Annen flatfisk, bunnfisk og dypvannsfisk",0.0,3441.0
10271,1505399,5.0,15.01.2018 19:16,70.573,17.227,-275,26163.0,Bunntrål,POK,Sei,MZZ,Annen marin fisk,"Annen flatfisk, bunnfisk og dypvannsfisk",0.0,3441.0
27484,1519324,28.0,02.02.2018 11:52,61.137,2.724,-289,26840.0,Bunntrål,POK,Sei,HAD,Hyse,Hyse,0.0,2475.0
27485,1519324,28.0,02.02.2018 11:52,61.137,2.724,-289,26840.0,Bunntrål,POK,Sei,HKE,Lysing,Annen torskefisk,0.0,2475.0


Ser også her at det er registrert forskjellige arter i 0-registrerningene på rundvekt. Det er da fornuftig å anta at dette enten er feilregistreringer eller fangst uten betydning.

In [107]:
# Fjerner radene med data nevnt over:
# Droppe tomme rader.
cleaned_df = filtered_df.copy().dropna()
# Konvertere positive Havdybdeverdier
cleaned_df['Havdybde stopp'] = cleaned_df['Havdybde stopp'].abs()
# Ta vekk havdybder på over 10 m, for å fjerne usannsynlige tråledybder.
cleaned_df = cleaned_df[cleaned_df['Havdybde stopp'] > 10]
# Droppe trekkavstand på over en viss lengde.
cleaned_df = cleaned_df[(cleaned_df['Trekkavstand'] > 0) & (cleaned_df['Trekkavstand'] < 50000)]
# Her kan vi kanskje undersøke

# Droppe ekstreme Bruttotonnasjeverdier.
cleaned_df = cleaned_df[(cleaned_df['Bruttotonnasje kombinert'] > 400) & (cleaned_df['Bruttotonnasje kombinert'] < 4000)]
# Droppe registreringer i områder som ikkje er i Norsk farvatn. Startposisjon med mindre enn 0.
cleaned_df = cleaned_df[cleaned_df['Stopposisjon lengde'] > 0]
cleaned_df = cleaned_df[cleaned_df['Stopposisjon bredde'] > 0]
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24833 entries, 93 to 305406
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Melding ID                24833 non-null  int64  
 1   Hovedområde start (kode)  24833 non-null  float64
 2   Stopptidspunkt            24833 non-null  object 
 3   Stopposisjon bredde       24833 non-null  float64
 4   Stopposisjon lengde       24833 non-null  float64
 5   Havdybde stopp            24833 non-null  int64  
 6   Trekkavstand              24833 non-null  float64
 7   Redskap FDIR              24833 non-null  object 
 8   Hovedart FAO (kode)       24833 non-null  object 
 9   Hovedart FAO              24833 non-null  object 
 10  Art FAO (kode)            24833 non-null  object 
 11  Art FAO                   24833 non-null  object 
 12  Art - gruppe              24833 non-null  object 
 13  Rundvekt                  24833 non-null  float64
 14  Bruttoton

Ser av value_counts at de mest vanlige redskapene er bunntrål og snurrevad. Her er det så mye data at vi fokuserer videre på dette. (Usikker på å inkludere snurrevad.

df['Redskap

In [108]:
# Slår sammen Bruttotonnasje 1969 og bruttotonnasje annen:
df.loc[:, 'Bruttotonnasje kombinert'] = df['Bruttotonnasje 1969'].fillna(df['Bruttotonnasje annen'])

In [109]:
# Kjører df.describe()
df.describe()

,Melding ID,Startposisjon bredde,Startposisjon lengde,Hovedområde start (kode),Lokasjon start (kode),Havdybde start,Varighet,Fangstår,Stopposisjon bredde,Stopposisjon lengde,...,Hovedart - FDIR (kode),Art - FDIR (kode),Art - gruppe (kode),Rundvekt,Lengdegruppe (kode),Bruttotonnasje 1969,Bruttotonnasje annen,Bredde,Fartøylengde,Bruttotonnasje kombinert
count,3.054340e+05,305434.000000,305434.000000,303433.000000,303433.000000,305434.000000,305434.000000,305434.000000,305434.000000,305434.000000,...,300456.000000,300452.000000,300452.000000,3.004560e+05,304750.000000,234005.000000,74774.000000,304750.000000,305434.000000,302463.000000
mean,1.658783e+06,66.278391,13.896474,14.463737,19.074712,-228.025292,537.095526,2017.999941,66.278198,13.917994,...,1326.729934,1414.625914,259.746585,7.438208e+03,4.575032,1408.386975,186.172573,10.274147,43.577660,1130.635982
std,9.130738e+04,8.547115,12.767469,13.001244,18.469340,226.062493,2201.624688,0.007677,8.562418,12.696535,...,614.506560,633.188386,320.124913,4.281086e+04,0.692769,1148.384145,165.761157,3.212845,18.440922,1135.802498
min,1.497177e+06,-73.840000,-179.517000,0.000000,0.000000,-5388.000000,0.000000,2017.000000,-73.831000,-116.792000,...,412.000000,211.000000,101.000000,0.000000e+00,3.000000,104.000000,21.000000,4.280000,15.010000,21.000000
25%,1.567228e+06,60.001000,4.727750,5.000000,7.000000,-273.000000,123.000000,2018.000000,59.982000,4.817000,...,1022.000000,1022.000000,201.000000,6.400000e+01,4.000000,496.000000,87.000000,7.600000,27.400000,267.000000
50%,1.674230e+06,68.516000,13.375000,8.000000,12.000000,-196.000000,296.000000,2018.000000,68.533000,13.347000,...,1032.000000,1032.000000,203.000000,3.000000e+02,5.000000,1184.000000,149.000000,9.500000,39.790000,567.000000
75%,1.735590e+06,71.437000,22.267000,20.000000,24.000000,-128.000000,494.000000,2018.000000,71.447000,22.307000,...,1038.000000,2202.000000,302.000000,2.236000e+03,5.000000,2053.000000,236.000000,13.000000,57.650000,1614.000000
max,1.800291e+06,89.983000,179.967000,81.000000,87.000000,1220.000000,125534.000000,2018.000000,89.267000,177.889000,...,6619.000000,6619.000000,9903.000000,1.100000e+06,5.000000,9432.000000,1147.000000,19.870000,133.880000,9432.000000


In [110]:
df.drop(columns=[#'Melding ID', Trengs muligens for å skille ulike tokter??
                 'Meldingstidspunkt', 
                 'Meldingsdato',
                 'Meldingsklokkeslett', 
                 'Starttidspunkt', 
                 'Startdato',
                 'Startklokkeslett', 
                 #'Startposisjon bredde', 
                 #'Startposisjon lengde',
                 #'Hovedområde start (kode)', 
                 'Hovedområde start',
                 'Lokasjon start (kode)', 
                 'Havdybde start', 
                 #'Stopptidspunkt', Det kan være interessant å se om det er sammenheng i tidspunkt på døgn og fangst.
                 'Stoppdato', 
                 'Stoppklokkeslett', 
                 'Varighet', 
                 'Fangstår',
                 'Stopposisjon bredde', 
                 'Stopposisjon lengde',
                 'Hovedområde stopp (kode)', 
                 'Hovedområde stopp',
                 'Lokasjon stopp (kode)', 
                 #'Havdybde stopp', 
                 #'Trekkavstand',
                 'Redskap FAO (kode)', 
                 'Redskap FAO', 
                 'Redskap FDIR (kode)',
                 #'Redskap FDIR', 
                 #'Hovedart FAO (kode)', 
                 #'Hovedart FAO',
                 'Hovedart - FDIR (kode)', 
                 #'Art FAO (kode)', 
                 #'Art FAO',
                 'Art - FDIR (kode)', 
                 'Art - FDIR', 
                 'Art - gruppe (kode)',
                 #'Art - gruppe', 
                 #'Rundvekt', 
                 #'Lengdegruppe (kode)', 
                 #'Lengdegruppe',
                 #'Bruttotonnasje kombinert'
                 'Bruttotonnasje 1969', 
                 'Bruttotonnasje annen', 
                 #'Bredde',
                 #'Fartøylengde'
                 ])
df

,Melding ID,Meldingstidspunkt,Meldingsdato,Meldingsklokkeslett,Starttidspunkt,Startdato,Startklokkeslett,Startposisjon bredde,Startposisjon lengde,Hovedområde start (kode),...,Art - gruppe (kode),Art - gruppe,Rundvekt,Lengdegruppe (kode),Lengdegruppe,Bruttotonnasje 1969,Bruttotonnasje annen,Bredde,Fartøylengde,Bruttotonnasje kombinert
0,1497177,01.01.2018,01.01.2018,00:00,31.12.2017,31.12.2017,00:00,-60.350,-46.133,NaN,...,506.0,Antarktisk krill,706714.0,5.0,28 m og over,9432.0,NaN,19.87,133.88,9432.0
1,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,74.885,16.048,20.0,...,202.0,Hyse,9594.0,5.0,28 m og over,1476.0,NaN,12.60,56.80,1476.0
2,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,74.885,16.048,20.0,...,201.0,Torsk,8510.0,5.0,28 m og over,1476.0,NaN,12.60,56.80,1476.0
3,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,74.885,16.048,20.0,...,301.0,Blåkveite,196.0,5.0,28 m og over,1476.0,NaN,12.60,56.80,1476.0
4,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,74.885,16.048,20.0,...,203.0,Sei,134.0,5.0,28 m og over,1476.0,NaN,12.60,56.80,1476.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305429,1800291,01.01.2019 09:28,01.01.2019,09:28,31.12.2018 19:41,31.12.2018,19:41,76.906,12.709,21.0,...,304.0,Steinbiter,145.0,5.0,28 m og over,1483.0,NaN,12.60,57.30,1483.0
305430,1800291,01.01.2019 09:28,01.01.2019,09:28,31.12.2018 19:41,31.12.2018,19:41,76.906,12.709,21.0,...,302.0,Uer,136.0,5.0,28 m og over,1483.0,NaN,12.60,57.30,1483.0
305431,1800291,01.01.2019 09:28,01.01.2019,09:28,31.12.2018 19:41,31.12.2018,19:41,76.906,12.709,21.0,...,304.0,Steinbiter,132.0,5.0,28 m og over,1483.0,NaN,12.60,57.30,1483.0
305432,1800291,01.01.2019 09:28,01.01.2019,09:28,31.12.2018 19:41,31.12.2018,19:41,76.906,12.709,21.0,...,302.0,Uer,102.0,5.0,28 m og over,1483.0,NaN,12.60,57.30,1483.0


In [111]:
# Antall registreringer med dei forskjellige Hovedarter.
print(df['Hovedart FAO'].value_counts())

Hovedart FAO
Torsk                   86215
Sei                     66311
Dypvannsreke            36977
Hyse                    28176
Lange                   13387
                        ...  
Sandflyndre                 3
Makrellstørje               2
Various squids nei *        2
Blåhval                     1
Annen laksefisk             1
Name: count, Length: 68, dtype: int64


In [112]:
# Få oversikt over hovedområder.
hovedområder = df.groupby(df['Hovedområde start'], as_index= False).agg(
    {'Hovedområde start': 'first', 'Hovedområde start (kode)': 'first'}
)
print(hovedområder)

                Hovedområde start  Hovedområde start (kode)
0                Admiralityfeltet                      16.0
1                        Bjørnøya                      20.0
2                   Britvinfeltet                      14.0
3                  Danmarkstredet                      61.0
4                 Eigersundbanken                       8.0
5                     Flemish Cap                      81.0
6                     Fyllas Bank                      68.0
7                     Færøybanken                      58.0
8                       Gammelock                      62.0
9                  Grønlandshavet                      26.0
10                     Gåsebanken                      11.0
11               Helgelandsbanken                       6.0
12                          Hopen                      23.0
13                    Kaninbanken                       1.0
14                 Murmanskkysten                       2.0
15         Nord av Horn  (Island)       

In [113]:
# Undersøker hvor det er flest aktiviteter
print(df['Hovedområde start'].value_counts())

Hovedområde start
Eigersundbanken                  45349
Røstbanken til Malangsgrunnen    29099
Vest-Finnmark                    27711
Øst-Finnmark                     26995
Bjørnøya                         25265
Shetland                         24813
Nordkappbanken                   18635
Vikingbanken                     16600
Skagerrak                        16097
Storegga-Frøyabanken             14884
Helgelandsbanken                  8591
Sentralbanken                     5584
Storbanken                        4823
Thor Iversens Bank                4551
Vestfjorden (Lofoten)             4469
Sentrale Nordsjø                  3387
Hopen                             3161
Vest av Skottland (Hebridene)     2008
Sørlige Norskehav                 1990
Østlige Norskehav                 1936
Admiralityfeltet                  1909
Skolpenbanken                     1904
Gåsebanken                        1699
Storfjord/Hinlopenstredet         1363
Vest-Spitsbergen                  1242
Vest av

In [114]:
# Undersøker hvilke arter som er de mest frekvente i Eigersundbanken.
eigersundbanken = df[df['Hovedområde start'] == 'Eigersundbanken']
eigersundbanken['Hovedart FAO'].value_counts()

Hovedart FAO
Dypvannsreke            15214
Sei                      8117
Øyepål                   4813
Torsk                    3586
Lysing                   3244
Kolmule                  3148
Breiflabb                1347
Tobis og annen sil       1177
Sild                     1105
Hestmakrell               850
Lange                     484
Hyse                      383
Lyr                       246
Strømsild                 199
Strømsild/Vassild         163
Rødspette                 161
Brosme                     81
Gråsteinbit                79
Sølvtorsk                  70
Sjøkreps                   69
Hvitting                   31
Stortare                   26
Makrell                    25
Skjellbrosme               21
Steinbiter                 20
Kveite                     17
Lomre                      15
Pigghå                     13
Annen torskefisk           12
Annen skate og rokke       12
Rognkjeks (felles)          8
Hakes nei. *                8
Flekksteinbit              

In [115]:
# Antall registreringer av fisk som ikkje er hovedart.
condition = df[(df['Hovedart FAO'] != df['Art FAO'])]
condition['Art FAO'].value_counts()

Art FAO
Torsk                   31512
Hyse                    31341
Sei                     23976
Lange                   14924
Uer (vanlig)            13906
                        ...  
Annen makrell               1
Various squids nei *        1
Taskekrabbe                 1
Sagtang                     1
Annen torskefisk            1
Name: count, Length: 110, dtype: int64

In [116]:
# Aktuelle redskaper å bruke: Bunntrål, Snurrevad og muligens Andre Liner. evt, dobbeltål
# Unngå garn.
# Aktuelle fisketyper å bruke hovedfangst: Sei
fish_types = ['Sei']
# Aktuelle Redskaper: Bunntrål, Snurrevad, Andre liner, (Dobbeltrål)
tools = ['Bunntrål']  # , 'Snurrevad', 'Andre liner'
# Aktuelle områder:
#areas = [x for x in range(0, 70)] # alle hovedområder minus antarktis og fiske i andre hav.
areas = [8, 42, 28, 7]

condition_3 = df['Hovedområde start (kode)'].isin(areas)
condition_1 = df['Hovedart FAO'].isin(fish_types) 
condition_2 = df['Redskap FDIR'].isin(tools)
combined_condition = condition_1 & condition_2 & condition_3
filtered_df = df[combined_condition].copy()
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21253 entries, 90 to 305299
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Melding ID                21253 non-null  int64  
 1   Meldingstidspunkt         21253 non-null  object 
 2   Meldingsdato              21253 non-null  object 
 3   Meldingsklokkeslett       21253 non-null  object 
 4   Starttidspunkt            21253 non-null  object 
 5   Startdato                 21253 non-null  object 
 6   Startklokkeslett          21253 non-null  object 
 7   Startposisjon bredde      21253 non-null  float64
 8   Startposisjon lengde      21253 non-null  float64
 9   Hovedområde start (kode)  21253 non-null  float64
 10  Hovedområde start         21253 non-null  object 
 11  Lokasjon start (kode)     21253 non-null  float64
 12  Havdybde start            21253 non-null  int64  
 13  Stopptidspunkt            21253 non-null  object 
 14  Stoppdato

In [117]:
filtered_df.loc[:, 'Bruttotonnasje kombinert'] = filtered_df['Bruttotonnasje 1969'].fillna(filtered_df['Bruttotonnasje annen'])

for start_stop in ['Startklokkeslett', 'Stoppklokkeslett']:
    # Konvertere klokkeslett til relevant verdi
    filtered_df[start_stop] = pd.to_datetime(filtered_df[start_stop], format='%H:%M').dt.strftime('%H-%M')
    # Konverter til Desimal-timer.
    filtered_df[f'{start_stop}_DecimalHours'] = filtered_df[start_stop].str.split('-').apply(lambda x: int(x[0]) + int(x[1]) / 60)
    # Konverter til Radianer.
    filtered_df[f'{start_stop}_Radians'] = filtered_df[f'{start_stop}_DecimalHours'] * (2 * np.pi / 24)
    # Regne ut sinus og cosinus.
    filtered_df[f'{start_stop}_TimeSin'] = np.sin(filtered_df[f'{start_stop}_Radians'])
    filtered_df[f'{start_stop}_TimeCos'] = np.cos(filtered_df[f'{start_stop}_Radians'])

In [118]:
filtered_df = filtered_df.drop(columns = [
    'Fangstår', 
    #'Melding ID', 
    'Meldingstidspunkt', 
    'Meldingsdato',
    'Meldingsklokkeslett', 
    'Starttidspunkt',
    'Startklokkeslett',
    #'Hovedområde start (kode)', 
    'Hovedområde start',
    'Lokasjon start (kode)', 
    'Havdybde start', 
    'Stopptidspunkt',
    'Varighet',
    'Stoppdato', 
    'Startposisjon bredde',
    'Startposisjon bredde',
    #'Stoppklokkeslett', 
    #'Stopposisjon bredde', 
    #'Stopposisjon lengde',
    'Hovedområde stopp (kode)', 
    'Hovedområde stopp',
    'Lokasjon stopp (kode)', 
    'Redskap FAO (kode)', 
    'Redskap FDIR (kode)',
    'Hovedart FAO (kode)',
    'Hovedart - FDIR (kode)', 
    'Art FAO (kode)', 
    #'Art FAO',
    'Art - FDIR (kode)', 
    'Art - FDIR', 
    'Art - gruppe (kode)',
    #'Art - gruppe', 
    'Lengdegruppe (kode)', 
    'Lengdegruppe',
    'Bredde',
    'Fartøylengde',
    'Bruttotonnasje 1969',
    'Bruttotonnasje annen'
    ])

In [119]:
def join_unique_values(series):
    return ', '.join(series.dropna().astype(str).unique())

# Gruppere df og slå sammen rader som kommer av samme aktivitet:
grouped_df = filtered_df.groupby(['Melding ID', 'Stoppklokkeslett'], as_index= False).agg({
    'Stopposisjon lengde': 'first',
    'Stopposisjon bredde': 'first',
    'Stoppklokkeslett': 'first',
    'Stoppklokkeslett_DecimalHours': 'first',
    'Stoppklokkeslett_Radians': 'first',
    'Stoppklokkeslett_TimeSin': 'first',
    'Hovedområde start (kode)': 'first',
    'Havdybde stopp': 'last',
    'Trekkavstand': 'first',  
    'Redskap FDIR': join_unique_values,
    'Hovedart FAO': 'first',  
    'Art FAO': join_unique_values,
    'Art - gruppe': join_unique_values,
    'Rundvekt': 'sum',   
    'Bruttotonnasje kombinert': 'first'
})
grouped_df

,Melding ID,Stopposisjon lengde,Stopposisjon bredde,Stoppklokkeslett,Stoppklokkeslett_DecimalHours,Stoppklokkeslett_Radians,Stoppklokkeslett_TimeSin,Hovedområde start (kode),Havdybde stopp,Trekkavstand,Redskap FDIR,Hovedart FAO,Art FAO,Art - gruppe,Rundvekt,Bruttotonnasje kombinert
0,1497341,2.134,61.278,03-13,3.216667,0.842121,0.746057,42.0,-211,29124.0,Bunntrål,Sei,Sei,Sei,30000.0,2053.0
1,1497341,1.982,61.343,08-37,8.616667,2.255838,0.774393,42.0,-158,1972.0,Bunntrål,Sei,Sei,Sei,17000.0,2053.0
2,1497341,1.702,61.552,14-06,14.100000,3.691371,-0.522499,42.0,-264,27885.0,Bunntrål,Sei,Sei,Sei,20000.0,2053.0
3,1497341,1.885,61.401,21-30,21.500000,5.628687,-0.608761,28.0,-180,16590.0,Bunntrål,Sei,Sei,Sei,3000.0,2053.0
4,1497843,5.664,63.401,22-19,22.316667,5.842490,-0.426569,7.0,-311,34192.0,Bunntrål,Sei,"Sei, Lysing, Lange","Sei, Annen torskefisk",1685.0,536.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5739,1800158,4.082,62.656,15-30,15.500000,4.057891,-0.793353,7.0,-191,2157.0,Bunntrål,Sei,"Sei, Lange, Torsk, Lyr","Sei, Annen torskefisk, Torsk",6860.0,2053.0
5740,1800158,4.802,62.778,19-56,19.933333,5.218534,-0.874620,7.0,-145,17655.0,Bunntrål,Sei,"Sei, Torsk","Sei, Torsk",650.0,2053.0
5741,1800172,5.418,63.251,00-44,0.733333,0.191986,0.190809,7.0,-249,9287.0,Bunntrål,Sei,"Sei, Lysing, Lange, Lyr","Sei, Annen torskefisk",180.0,567.0
5742,1800183,-0.609,61.494,00-00,0.000000,0.000000,0.000000,42.0,-172,32219.0,Bunntrål,Sei,"Sei, Hyse, Torsk, Lange, Lysing, Brosme","Sei, Hyse, Torsk, Annen torskefisk",1140.0,3909.0


In [120]:
# Droppe tomme rader.
cleaned_df = grouped_df.copy().dropna()
# Konvertere positive Havdybdeverdier
cleaned_df['Havdybde stopp'] = cleaned_df['Havdybde stopp'].abs()
# Ta vekk havdybder på over 10 m, for å fjerne usannsynlige tråledybder.
cleaned_df = cleaned_df[cleaned_df['Havdybde stopp'] > 10]
# Droppe trekkavstand på over ei viss lengd.
cleaned_df = cleaned_df[(cleaned_df['Trekkavstand'] > 10) & (cleaned_df['Trekkavstand'] < 60000)]

# Droppe ekstreme Bruttotonnasjeverdier. Under
cleaned_df = cleaned_df[(cleaned_df['Bruttotonnasje kombinert'] > 400) & (cleaned_df['Bruttotonnasje kombinert'] < 4000)]
# Droppe registreringer i områder som ikkje er i Norsk farvatn. Startposisjon med mindre enn 0.
cleaned_df = cleaned_df[cleaned_df['Stopposisjon lengde'] > 0]
cleaned_df = cleaned_df[cleaned_df['Stopposisjon bredde'] > 0]
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3889 entries, 0 to 5741
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Melding ID                     3889 non-null   int64  
 1   Stopposisjon lengde            3889 non-null   float64
 2   Stopposisjon bredde            3889 non-null   float64
 3   Stoppklokkeslett               3889 non-null   object 
 4   Stoppklokkeslett_DecimalHours  3889 non-null   float64
 5   Stoppklokkeslett_Radians       3889 non-null   float64
 6   Stoppklokkeslett_TimeSin       3889 non-null   float64
 7   Hovedområde start (kode)       3889 non-null   float64
 8   Havdybde stopp                 3889 non-null   int64  
 9   Trekkavstand                   3889 non-null   float64
 10  Redskap FDIR                   3889 non-null   object 
 11  Hovedart FAO                   3889 non-null   object 
 12  Art FAO                        3889 non-null   object

In [121]:
# Lager en liste med features
all_features = [
    'Stopposisjon lengde', 
    'Stopposisjon bredde', 
    'Stoppklokkeslett_Radians', 
    'Havdybde stopp', 
    'Trekkavstand',  
    'Rundvekt',
    'Bruttotonnasje kombinert'
]

cleaned_df[all_features]

,Stopposisjon lengde,Stopposisjon bredde,Stoppklokkeslett_Radians,Havdybde stopp,Trekkavstand,Rundvekt,Bruttotonnasje kombinert
0,2.134,61.278,0.842121,211,29124.0,30000.0,2053.0
1,1.982,61.343,2.255838,158,1972.0,17000.0,2053.0
2,1.702,61.552,3.691371,264,27885.0,20000.0,2053.0
3,1.885,61.401,5.628687,180,16590.0,3000.0,2053.0
4,5.664,63.401,5.842490,311,34192.0,1685.0,536.0
...,...,...,...,...,...,...,...
5737,5.417,63.151,5.057092,176,4119.0,7330.0,567.0
5738,4.093,62.654,2.474004,185,14460.0,3000.0,2053.0
5739,4.082,62.656,4.057891,191,2157.0,6860.0,2053.0
5740,4.802,62.778,5.218534,145,17655.0,650.0,2053.0


In [122]:
# Importerer verktøy for å kalkulere feil i modellen - gjennomsnittlig feil?
from sklearn.metrics import mean_absolute_error

# Lager en for-loop for å generere resultater for ulike target-values.
for predict in all_features:
    features = [f for f in all_features if f != predict]
    X = cleaned_df[features]
    y = cleaned_df[predict]

    # Definerer X og y, X er features, y er målverdien (target value)
    X = cleaned_df[features]
    y = cleaned_df[predict]

    # Splitter settet i trainsett og testsett - Da kan man sammenligne prediksjonene mot et test-sett
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

    # Velger MLmodell
    fishery_model = DecisionTreeRegressor(random_state=1)

    # Trener(?) modellen på train-settet
    fishery_model.fit(X_train, y_train)

    # Prediksjoner - predikerer tar X-val verdiene og predikerer rundvekt basert på disse tallene.
    val_predictions = fishery_model.predict(X_val)

    # Sammenligner et utvalg (X-val.head()) av prediksjoner mot relaterte y_val.head()
    print(f'--------- Target value: {predict} -------')
    print(fishery_model.predict(X_val.head()))
    print(y_val.head())

    # Tar inn y_val verdiene og regner ut gjennomsnittlig feil mellom prediksjonene og test-verdiene(val_y)
    val_mae = mean_absolute_error(y_val, val_predictions)
    print(f'\nMean absolute error: {val_mae}')
    print(f'\nScore: {fishery_model.score(X_val, y_val)}\n')


--------- Target value: Stopposisjon lengde -------
[5.601 2.968 3.191 5.401 1.614]
1154    5.567
3740    2.985
4175    3.179
2687    5.365
185     1.586
Name: Stopposisjon lengde, dtype: float64

Mean absolute error: 0.12744344473007713

Score: 0.9708409649668129

--------- Target value: Stopposisjon bredde -------
[62.733 60.485 59.655 63.11  61.68 ]
1154    62.735
3740    60.497
4175    59.546
2687    63.222
185     61.635
Name: Stopposisjon bredde, dtype: float64

Mean absolute error: 0.1537814910025707

Score: 0.8182904287801919

--------- Target value: Stoppklokkeslett_Radians -------
[0.01308997 3.08923278 5.86866961 0.01745329 2.79689013]
1154    2.530727
3740    4.886922
4175    1.042834
2687    5.183628
185     1.282817
Name: Stoppklokkeslett_Radians, dtype: float64

Mean absolute error: 2.0614907106147546

Score: -0.9926303460053112

--------- Target value: Havdybde stopp -------
[ 80. 113. 137. 247. 244.]
1154     89
3740    113
4175    138
2687    296
185     277
Name: Hav

In [123]:
for predict in all_features:
    features = [f for f in all_features if f != predict]
    X = cleaned_df[features]
    y = cleaned_df[predict]

    # Definerer X og y, X er features, y er målverdien (target value)
    X = cleaned_df[features]
    y = cleaned_df[predict]

    # Splitter settet i trainsett og testsett - Da kan man sammenligne prediksjonene mot et test-sett
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

    # Velger MLmodell
    fishery_model = linear_model.LinearRegression()

    # Trener(?) modellen på train-settet
    fishery_model.fit(X_train, y_train)

    # Prediksjoner - predikerer tar X-val verdiene og predikerer rundvekt basert på disse tallene.
    val_predictions = fishery_model.predict(X_val)

    # Sammenligner et utvalg (X-val.head()) av prediksjoner mot relaterte y_val.head()
    print(f'--------- Target value: {predict} -------')
    print(fishery_model.predict(X_val.head()))
    print(y_val.head())

    # Tar inn y_val verdiene og regner ut gjennomsnittlig feil mellom prediksjonene og test-verdiene(val_y)
    val_mae = mean_absolute_error(y_val, val_predictions)
    print(f'\nMean absolute error: {val_mae}')
    print(f'\nScore: {fishery_model.score(X_val, y_val)}\n')

--------- Target value: Stopposisjon lengde -------
[5.74487152 4.05586829 2.56093869 3.01273609 2.27730655]
1154    5.567
3740    2.985
4175    3.179
2687    5.365
185     1.586
Name: Stopposisjon lengde, dtype: float64

Mean absolute error: 1.0623234127851642

Score: 0.36408984593499305

--------- Target value: Stopposisjon bredde -------
[61.97829529 61.04372911 60.74440462 63.66076214 61.86259817]
1154    62.735
3740    60.497
4175    59.546
2687    63.222
185     61.635
Name: Stopposisjon bredde, dtype: float64

Mean absolute error: 0.9333871471614158

Score: 0.2655660617371204

--------- Target value: Stoppklokkeslett_Radians -------
[3.00546345 2.96434221 3.28146471 2.93284063 2.91050448]
1154    2.530727
3740    4.886922
4175    1.042834
2687    5.183628
185     1.282817
Name: Stoppklokkeslett_Radians, dtype: float64

Mean absolute error: 1.552494529114616

Score: 0.005107744166126915

--------- Target value: Havdybde stopp -------
[155.12013152 166.08366247 153.11320383 160.32